In [6]:
import json
import math
import subprocess
import shlex
from typing import List, Dict, Any, Optional, Tuple
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from langchain.llms import Ollama


In [7]:
# Choose a sentence-transformers model available locally or will auto-download.
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"  # fast and effective; change if you prefer

print("Loading embedding model:", EMBEDDING_MODEL_NAME)
embed_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
print("Embedding model loaded.")


Loading embedding model: all-MiniLM-L6-v2
Embedding model loaded.


In [8]:
def get_embedding(text: str) -> np.ndarray:
    if text is None:
        text = ""
    # SentenceTransformer returns numpy arrays
    emb = embed_model.encode([text], show_progress_bar=False, normalize_embeddings=True)
    return np.asarray(emb[0])

def cosine(a: np.ndarray, b: np.ndarray) -> float:
    if a is None or b is None:
        return 0.0
    # embeddings normalized in get_embedding; but compute robustly
    return float(np.clip(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-12), -1.0, 1.0))

def sim_to_01(sim: float) -> float:
    # map cosine similarity (-1..1) to 0..1
    return max(0.0, min(1.0, (sim + 1.0) / 2.0))


In [9]:
def step_text(step: Dict[str, Any]) -> str:
    """
    Extract a canonical text representation for a ToT step.
    Assumes step dict has 'input' and 'output' fields (based on your inference.py).
    """
    parts = []
    if step.get("input"):
        parts.append(str(step["input"]).strip())
    if step.get("output"):
        parts.append(str(step["output"]).strip())
    return " ".join(parts).strip()


In [10]:
def compute_goal_alignment(prompt: str, final_subtasks: List[Dict[str, Any]], use_last_only: bool = True) -> float:
    """
    Compute semantic similarity between the prompt (goal) and the ToT final answer.
    If use_last_only: only consider the last step output, else concatenate all outputs.
    Returns normalized score in [0,1].
    """
    if not final_subtasks:
        return 0.0
    if use_last_only:
        final_text = step_text(final_subtasks[-1])
    else:
        final_text = " ".join([step_text(s) for s in final_subtasks])
    if not final_text:
        return 0.0
    sim = cosine(get_embedding(final_text), get_embedding(prompt))
    return sim_to_01(sim)

def compute_stepwise_coherence(final_subtasks: List[Dict[str, Any]]) -> float:
    """
    Average adjacent-step similarity (normalized to 0..1).
    If only one step, returns 1.0 (perfect local coherence).
    """
    texts = [step_text(s) for s in final_subtasks]
    embs = [get_embedding(t) for t in texts if t]
    if len(embs) <= 1:
        return 1.0
    sims = []
    for i in range(len(embs) - 1):
        sims.append(sim_to_01(cosine(embs[i], embs[i+1])))
    return float(sum(sims) / len(sims))


In [11]:
def call_verifier_coverage(prompt: str, final_subtasks: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Uses ollama LLM to ask for missing concepts / expected concept count.
    The LLM is instructed to output strict JSON:
    {
      "missing": ["concept1", "concept2", ...],
      "expected_concepts": 4,
      "notes": "short text"
    }
    """
    # Build a compact representation of steps
    steps_text = "\n".join([f"Step {i+1}: {step_text(s)}" for i, s in enumerate(final_subtasks)])
    verifier_prompt = f"""
You are a verifier assistant. Given the original prompt and the chain-of-thought steps, output a strict JSON object with fields:
 - missing: a list of missing sub-concepts or checks that should be present for a complete solution (may be empty list).
 - expected_concepts: integer estimate for the number of major sub-concepts that should be covered.
 - notes: one-line diagnostic advice.

Respond ONLY with JSON (no extra commentary).

Original prompt:
\"\"\"{prompt}\"\"\"

Chain-of-thought steps:
\"\"\"{steps_text}\"\"\"

Produce the JSON now.
"""
    llm = Ollama(model='llama3.2')
    raw = llm(verifier_prompt)
    # Try to extract JSON substring (some LLMs may wrap). We'll attempt to find the first '{' and last '}'.
    try:
        start = raw.index("{")
        end = raw.rindex("}") + 1
        json_text = raw[start:end]
        parsed = json.loads(json_text)
        # sanitise fields
        parsed.setdefault("missing", [])
        parsed.setdefault("expected_concepts", max(1, len(final_subtasks)))
        parsed.setdefault("notes", "")
        return parsed
    except Exception as e:
        # Fallback heuristic: no LLM JSON available, return heuristics
        print("Warning: verifier did not return clean JSON. Falling back to heuristic coverage. Error:", e)
        # Simple heuristic: expected = len(steps), missing = []
        return {"missing": [], "expected_concepts": max(1, len(final_subtasks)), "notes": "fallback heuristic used"}


In [12]:
def compute_granularity(final_subtasks: List[Dict[str, Any]], ideal_len: int = 25, sigma: float = 12.0) -> float:
    """
    Score each step by how close its token count (word count) is to ideal_len using a Gaussian mapping.
    Returns average score in [0,1].
    """
    lengths = []
    for s in final_subtasks:
        l = len(step_text(s).split())
        lengths.append(l)
    if not lengths:
        return 1.0
    scores = [math.exp(-((l - ideal_len) ** 2) / (2 * (sigma ** 2))) for l in lengths]
    # normalize scores to [0,1] (they already are because exp -> (0,1])
    return float(sum(scores) / len(scores))

def compute_redundancy(final_subtasks: List[Dict[str, Any]], cluster_threshold: float = 0.85) -> float:
    """
    Greedy clustering by cosine similarity threshold. Returns "uniqueness" fraction (higher = less redundant).
    """
    texts = [step_text(s) for s in final_subtasks]
    if not texts:
        return 1.0
    embs = np.vstack([get_embedding(t) for t in texts])
    # Use AgglomerativeClustering with distance threshold equivalent to (1 - cluster_threshold)
    try:
        # sklearn AgglomerativeClustering with distance_threshold requires n_clusters=None
        clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.0 - cluster_threshold, linkage="average").fit(embs)
        labels = clustering.labels_
        num_clusters = len(set(labels))
    except Exception:
        # fallback: greedy grouping
        centroids = []
        labels = []
        for e in embs:
            placed = False
            for i, c in enumerate(centroids):
                if cosine(e, c) >= cluster_threshold:
                    # update centroid (mean)
                    centroids[i] = (centroids[i] * labels.count(i) + e) / (labels.count(i) + 1)
                    labels.append(i)
                    placed = True
                    break
            if not placed:
                centroids.append(e)
                labels.append(len(centroids) - 1)
        num_clusters = len(centroids)
    unique_frac = num_clusters / max(1, len(texts))
    # uniqueness score: closer to 1 is better (less redundancy)
    return float(unique_frac)


In [14]:
def combined_reasoning_score(out: Dict[str, Any], prompt: str, weights: Optional[Dict[str, float]] = None) -> Dict[str, Any]:
    """
    Compute component scores and combined reasoning score.
    out: dictionary returned by run_inference_pipeline (expects out['final_subtasks'] list)
    prompt: original prompt text
    """
    final_subtasks = out.get("final_subtasks", [])
    # components
    alignment = compute_goal_alignment(prompt, final_subtasks, use_last_only=True)
    coherence = compute_stepwise_coherence(final_subtasks)
    granularity = compute_granularity(final_subtasks)
    redundancy = compute_redundancy(final_subtasks)
    # coverage uses LLM verifier
    verifier_info = call_verifier_coverage(prompt, final_subtasks)
    missing = verifier_info.get("missing", [])
    expected = max(1, int(verifier_info.get("expected_concepts", max(1, len(final_subtasks)))))
    coverage = max(0.0, 1.0 - (len(missing) / expected))
    # default weights (sum to 1)
    if weights is None:
        weights = {
            "alignment": 0.30,
            "coherence": 0.25,
            "coverage": 0.25,
            "granularity": 0.10,
            "redundancy": 0.10
        }
    # ensure normalization
    s = sum(weights.values())
    if s == 0:
        raise ValueError("weights must sum to > 0")
    for k in weights:
        weights[k] = weights[k] / s
    reasoning_score = (
        weights["alignment"] * alignment
        + weights["coherence"] * coherence
        + weights["coverage"] * coverage
        + weights["granularity"] * granularity
        + weights["redundancy"] * redundancy
    )
    # diagnostics
    diagnostics = {
        "stepwise_similarities": None,  # compute adjacent sims for debugging
        "missing_concepts": missing,
        "expected_concepts": expected,
        "average_step_length": None,
        "unique_clusters": None,
        "num_steps": len(final_subtasks),
        "verifier_notes": verifier_info.get("notes", "")
    }
    # populate additional diagnostics
    # adjacent sims
    texts = [step_text(s) for s in final_subtasks]
    embs = [get_embedding(t) for t in texts if t]
    adj_sims = [sim_to_01(cosine(embs[i], embs[i+1])) for i in range(len(embs)-1)] if len(embs) > 1 else []
    diagnostics["stepwise_similarities"] = adj_sims
    diagnostics["average_step_length"] = float(sum(len(t.split()) for t in texts) / max(1, len(texts)))
    # clusters uniqueness
    diagnostics["unique_clusters"] = float(compute_redundancy(final_subtasks))
    # verdict
    if reasoning_score >= 0.75:
        verdict = "Reasoning is coherent and well-aligned with goal."
    elif reasoning_score >= 0.5:
        verdict = "Reasoning is partially coherent; some gaps or inconsistencies."
    else:
        verdict = "Reasoning shows substantial drift or incompleteness."
    return {
        "reasoning_score": float(reasoning_score),
        "components": {
            "goal_alignment": float(alignment),
            "stepwise_coherence": float(coherence),
            "coverage": float(coverage),
            "granularity": float(granularity),
            "redundancy": float(redundancy)
        },
        "diagnostics": diagnostics,
        "verdict": verdict
    }


In [16]:
# Example synthetic `out` structure (adapt to your actual run_inference_pipeline output)
example_out = {
    "final_subtasks": [
        {"id": 1, "input": "Understand the problem: find sum of list", "output": "We need to sum numeric elements; check for non-numeric values."},
        {"id": 2, "input": "", "output": "Filter out non-numeric entries and convert strings to floats."},
        {"id": 3, "input": "", "output": "Use a running sum; consider edge cases: empty list, None entries."},
        {"id": 4, "input": "", "output": "Return the sum as float; if empty list return 0."}
    ],
    # optionally other fields your pipeline returns
}

prompt = "Write a robust plan to compute the sum of a list of items that may include strings and None."
print("Running semantic scorer on example output...")
result = combined_reasoning_score(example_out, prompt)
print(json.dumps(result, indent=2))


Running semantic scorer on example output...


C:\Users\amanc\AppData\Local\Temp\ipykernel_13992\2904521226.py:29: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model='llama3.2')
C:\Users\amanc\AppData\Local\Temp\ipykernel_13992\2904521226.py:30: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  raw = llm(verifier_prompt)


{
  "reasoning_score": 0.6414356923762288,
  "components": {
    "goal_alignment": 0.7514747381210327,
    "stepwise_coherence": 0.7130378832419714,
    "coverage": 0.33333333333333337,
    "granularity": 0.5440046679609276,
    "redundancy": 1.0
  },
  "diagnostics": {
    "stepwise_similarities": [
      0.6731556951999664,
      0.631446972489357,
      0.8345109820365906
    ],
    "missing_concepts": [
      "Handle non-numeric values",
      "Edge case handling for empty list"
    ],
    "expected_concepts": 3,
    "average_step_length": 11.75,
    "unique_clusters": 1.0,
    "num_steps": 4,
    "verifier_notes": "Consider using try/except blocks to catch non-numeric value errors and handle edge cases correctly."
  },
  "verdict": "Reasoning is partially coherent; some gaps or inconsistencies."
}


In [19]:
from inference import run_inference_pipeline
import json

example_prompt = "how can i solve the question (a^2 - b^2)?"
out = run_inference_pipeline(example_prompt,auto_extend=True)

print("\n=== FORMATTED FINAL SUBTASKS ===")
for i, s in enumerate(out["final_subtasks"], start=1):
    text = s.get("input") or s.get("output") or ""
    print(f"{i}. {text}")


Subtasks:
 1. Recall the formula for difference of squares: (a-b)(a+b)
 2. Identify values of a and b in the given equation
 3. Apply the formula to simplify the expression
 4. Solve for the resulting expression if necessary
[DEBUG] run_decomposer returned type=<class 'list'>, preview=['Recall the formula for difference of squares: (a-b)(a+b)', 'Identify values of a and b in the given equation', 'Apply the formula to simplify the expression', 'Solve for the resulting expression if necessary']

=== FORMATTED FINAL SUBTASKS ===
1. Recall the formula for difference of squares: (a-b)(a+b)
2. Identify values of a and b in the given equation
3. Apply the formula to simplify the expression
4. Solve for the resulting expression if necessary


In [20]:
reasoning_result = combined_reasoning_score(out, example_prompt)
print("\n=== REASONING DIAGNOSTIC ===")
print(json.dumps(reasoning_result, indent=2))


=== REASONING DIAGNOSTIC ===
{
  "reasoning_score": 0.7063295120804355,
  "components": {
    "goal_alignment": 0.7272500991821289,
    "stepwise_coherence": 0.7385770032803217,
    "coverage": 0.6666666666666667,
    "granularity": 0.3684356483904969,
    "redundancy": 1.0
  },
  "diagnostics": {
    "stepwise_similarities": [
      0.7189392745494843,
      0.6485119313001633,
      0.8482798039913177
    ],
    "missing_concepts": [
      "values of a and b"
    ],
    "expected_concepts": 3,
    "average_step_length": 8.0,
    "unique_clusters": 1.0,
    "num_steps": 4,
    "verifier_notes": "Clearly identify a and b in the equation (a^2 - b^2)"
  },
  "verdict": "Reasoning is partially coherent; some gaps or inconsistencies."
}
